## IMPORTING THE REQUIRED LIBRARIES

In [1]:
import pandas as pd
import requests as re
from getpass import getpass
from IPython.display import display

## GENERATING A FUNCTION THAT GIVES US THE DATES BETWEEN TWO DATES

In [2]:
def get_dates(start_date, end_date):
    return pd.Series(pd.date_range(start_date, end_date, freq = 'd').format())

## GETTING THE CITY CODE OF A GIVEN CITY NAME

In [3]:
rapidAPIkey = getpass()
url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/autosuggest/v1.0/US/USD/en-US/"
headers = {'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com", 'x-rapidapi-key': rapidAPIkey}

def get_city_code(city, url, headers):

    parameters = {"query": city}
    response = re.get(url, headers = headers, params = parameters)

    return response.json()["Places"][0]["PlaceId"]

## GENERATING A FUNCTION THAT RETURNS THE PRICE, THE NAMES OF THE AIRPORTS FOR THE PLACES OF ORIGIN AND ARRIVAL AND THE COMPANY BETWEEN 2021-11-01 AND 2021-11-15 - NEW YORK TO SAN FRANCISCO

In [7]:
city_departure = 'New York'
city_arrival = 'San Francisco'

dates = get_dates("2021-11-01", "2021-11-15")
departure_code = get_city_code(city_departure, url, headers)
arrival_code = get_city_code(city_arrival, url, headers)
new_url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US"

def get_data(dates, departure_code, arrival_code, url, headers):

    prices = list()
    departures_airports = list()
    arrivals_airports = list()
    companies = list()

    for date in dates:

        parameters = {"inboundpartialdate": {date}}
        query_url = f'{url}/{departure_code}/{arrival_code}/{date}'
        response = re.get(query_url, headers = headers, params = parameters)

        prices.append(response.json()['Quotes'][0]['MinPrice'])
        departures_airports.append(response.json()['Places'][0]['Name'])
        companies.append(response.json()['Carriers'][0]['Name'])

        if city_departure in response.json()['Places'][1]['Name']:
            arrivals_airports.append(response.json()['Places'][2]['Name'])

        else:
            arrivals_airports.append(response.json()['Places'][1]['Name'])
            

    return pd.DataFrame({'date': dates, 'company': companies, 'departure': departures_airports, 
                         'arrival': arrivals_airports, 'price': prices})

flights_data = get_data(dates, departure_code, arrival_code, new_url, headers)

## DISPLAYING THE RESULTING DATAFRAME

In [8]:
display(flights_data)

,date,company,departure,arrival,price
0,2021-11-01,Alaska Airlines,New York Newark,San Francisco International,68
1,2021-11-02,jetBlue,New York John F. Kennedy,San Francisco International,76
2,2021-11-03,jetBlue,New York John F. Kennedy,San Francisco International,76
3,2021-11-04,Alaska Airlines,New York John F. Kennedy,San Francisco International,68
4,2021-11-05,Alaska Airlines,New York Newark,San Francisco International,68
5,2021-11-06,Alaska Airlines,New York John F. Kennedy,San Francisco International,67
6,2021-11-07,Alaska Airlines,New York Newark,San Francisco International,68
7,2021-11-08,Alaska Airlines,New York John F. Kennedy,San Francisco International,75
8,2021-11-09,Frontier Airlines,New York Newark,San Francisco International,66
9,2021-11-10,Frontier Airlines,New York Newark,San Francisco International,64
